In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import re
import scipy
import itertools
import pickle
import time
import os.path
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.cm import ScalarMappable, get_cmap
from matplotlib.colors import Normalize
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF as sm_ECDF
import statsmodels.formula.api as smf
from statsmodels.genmod.bayes_mixed_glm import BinomialBayesMixedGLM
import concurrent.futures
import attila_utils
import gem_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ar = gem_tools.read_active_reactions(groupdict={'m-control': ('all_control', 'MSBB'), 'm-AD-B2': ('SubtypeB2_AD', 'MSBB')})

In [4]:
%connect_info

{
  "shell_port": 32997,
  "iopub_port": 51271,
  "stdin_port": 43553,
  "control_port": 60079,
  "hb_port": 40367,
  "ip": "127.0.0.1",
  "key": "8c86592b-3a5eda9a84621ca9d2d1182c",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-24b37cc4-ab81-4670-a0ce-90a1a77107c8.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
